In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

D:\Anaconda\envs\GPU_Pytorchpy39\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
transform = transforms.Compose(
    [torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))])

batch_size = 6000

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

In [3]:
from mnist import *
from viz import *

import torch
from torch.nn import Sequential, Module, CrossEntropyLoss
from torch.nn.functional import normalize
import numpy as np
from neurophoxTorch.torch import RMTorch
from scipy.stats import unitary_group
from tqdm import tqdm_notebook as pbar
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
def rc_mul(real: torch.Tensor, comp: torch.Tensor):
    return real.unsqueeze(dim=0) * comp


def cc_mul(comp1: torch.Tensor, comp2: torch.Tensor) -> torch.Tensor:
    real = comp1[0] * comp2[0] - comp1[1] * comp2[1]
    comp = comp1[0] * comp2[1] + comp1[1] * comp2[0]
    return torch.stack((real, comp), dim=0)

def phasor(real: torch.Tensor):
    return torch.stack((real.cos(), real.sin()), dim=0)


def cnorm(comp: torch.Tensor):
    return (comp[0] ** 2 + comp[1] ** 2).sqrt()


def cnormsq(comp: torch.Tensor):
    return comp[0] ** 2 + comp[1] ** 2


def to_complex_t(nparray: np.ndarray):
    return torch.stack((torch.as_tensor(nparray.real),
                        torch.as_tensor(nparray.imag)), dim=0)


class ElectroopticNonlinearity(Module):
    def __init__(self, alpha: float=0.1, g: float=0.05 * np.pi, phi_b: float=np.pi):
        super(ElectroopticNonlinearity, self).__init__()
        self.alpha = alpha
        self.g = g
        self.phi_b = phi_b

    def forward(self, inputs):
        phase = 0.5 * self.g * cnormsq(inputs) + 0.5 * self.phi_b
        return np.sqrt(1 - self.alpha) * cc_mul(rc_mul(phase.cos(), phasor(-phase)), inputs)


class CNormSq(Module):
    def __init__(self, normed=True):
        super(CNormSq, self).__init__()
        self.normed = normed

    def forward(self, inputs):
        return normalize(cnormsq(inputs), dim=1) if self.normed else cnormsq(inputs)

D:\ResearchProject\PNNRobustOptimization\OpenSourceCodes\System3-MZI Mesh\mnist.py:2: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  import scipy as sp


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 64)
        self.norm1 = nn.BatchNorm2d(6)
        self.norm2 = nn.BatchNorm2d(16)
        self.norm3 = nn.BatchNorm1d(120)
        self.layer1 = RMTorch(64, phase_error = 0, phase_error_files = None,bs_error=0,bs_error_files = None)
        self.output = CNormSq()

    def forward(self,x):

        x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
        x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.norm3(x)
        x = self.fc2(x)
        x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
        x,_,_,_ = self.layer1(x)
        x = self.output(x)
        self.at_sensor_intensity = x
        return self.at_sensor_intensity[:,:10]
    
    def phy_forward(self,x):
        self.in_outs_phy = []
        with torch.no_grad():
            x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
            x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
            x = x.view(x.size(0), -1) 
            x = self.fc1(x)
            x = self.norm3(x)
            x = self.fc2(x)
            x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
            x, _, _, _ = self.layer1(x)
            x = self.output(x)
            self.at_sensor_intensity_phy = x
        return self.at_sensor_intensity_phy[:,:10]    
    
    def phy_replace_sim(self):
        # PAT: replace the output
        with torch.no_grad():
            self.at_sensor_intensity.data.copy_(self.at_sensor_intensity_phy.data)    

    def apply_constraints(self):
        self.conv1.cuda().weight.data = torch.clamp(self.conv1.cuda().weight.data, -1, 1)
        self.conv2.cuda().weight.data = torch.clamp(self.conv2.cuda().weight.data, -1, 1)
        self.fc1.cuda().weight.data = torch.clamp(self.fc1.cuda().weight.data, -1, 1)
        self.fc2.cuda().weight.data = torch.clamp(self.fc2.cuda().weight.data, -1, 1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net = LeNet5()
net.to(device)

cuda:0


LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (norm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): RMTorch()
  (output): CNormSq()
)

In [15]:
### Store the ideal parameters
### nenn,nepn,nenp,nepp indicate the parameters without fabrication errors
### nenn,nepn,nenp,nepp indicate the parameters with    fabrication errors
[nenn, nepn, nenp, nepp], [enn, epn, enp, epp] = net.layer1.mesh_model.mzi_error_tensors()
[nenn_init, nepn_init, nenp_init, nepp_init] = [nenn, nepn, nenp, nepp] ### And we store the nenn... first

In [16]:
bsE_scale = 0.15
phaseE_scale = 0.15

In [17]:
### Introduce manufacturing errors
net.layer1.mesh_model.bs_error = bsE_scale
[nenn, nepn, nenp, nepp], [enn, epn, enp, epp] = net.layer1.mesh_model.mzi_error_tensors()
net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp, dtype=torch.float32).cuda()
net.layer1.enn, net.layer1.epn, net.layer1.enp, net.layer1.epp = \
                    torch.as_tensor(enn, dtype=torch.float32).cuda(), \
                        torch.as_tensor(epn, dtype=torch.float32).cuda(), \
                    torch.as_tensor(enp, dtype=torch.float32).cuda(), \
                        torch.as_tensor(epp, dtype=torch.float32).cuda()

### You can save the errors and directely load if you need
# np.save('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/enn.npy', enn)
# np.save('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/epn.npy', epn)
# np.save('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/enp.npy', enp)
# np.save('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/epp.npy', epp)
enn = np.load('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/enn.npy')
epn = np.load('.\MZI_Error/N64/bsE'+str(bsE_scale)+'/epn.npy')
enp = np.load('./MZI_Error/N64/bsE'+str(bsE_scale)+'/enp.npy')
epp = np.load('./MZI_Error/N64/bsE'+str(bsE_scale)+'/epp.npy')

[nenn_n, nepn_n, nenp_n, nepp_n] = [enn, epn, enp, epp] ### Here, we copy the generated errors to nenn...

In [18]:
### Introduce phase errors
b0_theta_error = torch.randn(net.layer1.theta.shape).cuda() * phaseE_scale
b0_phi_error = torch.randn(net.layer1.phi.shape).cuda() * phaseE_scale

### You can save the errors and directely load if you need
# np.save('.\MZI_Error/N64/phaseE'+str(phaseE_scale)+'/theta.npy', b0_theta_error.cpu().detach().numpy())
# np.save('.\MZI_Error/N64/phaseE'+str(phaseE_scale)+'/phi.npy', b0_phi_error.cpu().detach().numpy())
b0_theta_error = np.load('.\MZI_Error/N64/phaseE'+str(phaseE_scale)+'/theta.npy')
b0_phi_error = np.load('.\MZI_Error/N64/phaseE'+str(phaseE_scale)+'/phi.npy')
b0_theta_error = torch.tensor(b0_theta_error).to(device)
b0_phi_error = torch.tensor(b0_phi_error).to(device)

In [21]:
import random
import numpy as np
import torch
def set_random_seed(seed: int = 42, deterministic: bool = True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)          
        torch.cuda.manual_seed_all(seed)     
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_random_seed(seed=42)

Now we start to compare different methods

### 01-Standard BP

In [22]:
import torch.optim as optim
net = LeNet5()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 400, gamma = 0.5)

In [23]:
import time
net.to(device)
start = time.time()
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs_sim = net(inputs)
        loss0 = criterion(outputs_sim, labels)
        loss = loss0
        loss.backward()
        optimizer.step()
        net.apply_constraints()
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f} Classification loss:{loss0.cpu().detach().numpy():.3f}')
            running_loss = 0.0
    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error  
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda()   
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
        accu_wo_n = 100 * correct / total
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] loss: 2.303 Classification loss:2.294
[1,     4] loss: 2.204 Classification loss:2.163
[1,     6] loss: 2.050 Classification loss:2.015
[1,     8] loss: 1.928 Classification loss:1.903
[1,    10] loss: 1.855 Classification loss:1.839
[2,     2] loss: 1.803 Classification loss:1.790
[2,     4] loss: 1.761 Classification loss:1.754
[2,     6] loss: 1.737 Classification loss:1.731
[2,     8] loss: 1.716 Classification loss:1.714
[2,    10] loss: 1.700 Classification loss:1.696
[3,     2] loss: 1.682 Classification loss:1.678
[3,     4] loss: 1.671 Classification loss:1.670
[3,     6] loss: 1.662 Classification loss:1.658
[3,     8] loss: 1.651 Classification loss:1.648
[3,    10] loss: 1.644 Classification loss:1.639
[4,     2] loss: 1.636 Classification loss:1.636
[4,     4] loss: 1.628 Classification loss:1.627
[4,     6] loss: 1.623 Classification loss:1.623
[4,     8] loss: 1.618 Classification loss:1.617
[4,    10] loss: 1.613 Classification loss:1.609
[5,     2] loss: 1.6

In [24]:
PATH = './Training Results/01-Standard BP.pth'
torch.save(net.state_dict(), PATH)
PATH = ''

### 02-PAT

In [25]:
import torch.optim as optim
net = LeNet5()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 400, gamma = 0.5)

In [26]:
import time
net.to(device)
start = time.time()
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs_sim = net(inputs)
        with torch.no_grad():
            net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_n, dtype=torch.float32).cuda()
            outputs_phy = net.phy_forward(inputs)
            net.phy_replace_sim()
            outputs_sim.data.copy_(outputs_phy.data)
            net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data - b0_phi_error  
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_init, dtype=torch.float32).cuda()            
        
        # loss0 = criterion(outputs, labels)
        loss0 = criterion(outputs_sim, labels)
        loss = loss0
        loss.backward()
        optimizer.step()
        net.apply_constraints()
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f} Classification loss:{loss0.cpu().detach().numpy():.3f}')
            running_loss = 0.0
    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error  
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda()   
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
        accu_wo_n = 100 * correct / total
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] loss: 2.291 Classification loss:2.285
[1,     4] loss: 2.258 Classification loss:2.247
[1,     6] loss: 2.226 Classification loss:2.218
[1,     8] loss: 2.197 Classification loss:2.190
[1,    10] loss: 2.180 Classification loss:2.175
[2,     2] loss: 2.161 Classification loss:2.159
[2,     4] loss: 2.147 Classification loss:2.143
[2,     6] loss: 2.139 Classification loss:2.136
[2,     8] loss: 2.128 Classification loss:2.129
[2,    10] loss: 2.117 Classification loss:2.111
[3,     2] loss: 2.108 Classification loss:2.109
[3,     4] loss: 2.105 Classification loss:2.107
[3,     6] loss: 2.098 Classification loss:2.096
[3,     8] loss: 2.099 Classification loss:2.096
[3,    10] loss: 2.092 Classification loss:2.093
[4,     2] loss: 2.088 Classification loss:2.087
[4,     4] loss: 2.082 Classification loss:2.083
[4,     6] loss: 2.078 Classification loss:2.078
[4,     8] loss: 2.074 Classification loss:2.073
[4,    10] loss: 2.070 Classification loss:2.067
[5,     2] loss: 2.0

In [27]:
PATH = './Training Results/02-PAT.pth'
torch.save(net.state_dict(), PATH)
PATH = ''

### 03-SAT-In silico

In [28]:
from sam import SAM
import torch.optim as optim
net = LeNet5()
net.to(device)
criterion = nn.CrossEntropyLoss()
base_optimizer = torch.optim.Adam
optimizer = SAM(net.parameters(), rho=0.3, base_optimizer=base_optimizer,lr=1e-1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 400, gamma = 0.5)

In [29]:
import time
net.to(device)
start = time.time()
for epoch in range(40):  # loop over the dataset multiple times
### You can consider increasing the training epochs if the accuracy does not converge
### I find the different initialization results may lead to different training epochs, sometimes short, sometimes long
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # forward + backward + optimize
        outputs_sim = net(inputs)
        loss0 = criterion(outputs_sim, labels)

        loss = loss0
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        
        
        outputs_sim_SAM = net(inputs)
        lossSAM0 = criterion(outputs_sim_SAM, labels)

        lossSAM = lossSAM0
        lossSAM.backward()
        optimizer.second_step(zero_grad=True) 
        # net.apply_constraints()
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f}')
            running_loss = 0.0

    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                    
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error 
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda() 
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] loss: 2.313
[1,     4] loss: 2.306
[1,     6] loss: 2.289
[1,     8] loss: 2.241
[1,    10] loss: 2.175


D:\Anaconda\envs\GPU_Pytorchpy39\lib\site-packages\torch\optim\lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[2,     2] loss: 2.103
[2,     4] loss: 2.055
[2,     6] loss: 1.954
[2,     8] loss: 1.898
[2,    10] loss: 1.841
[3,     2] loss: 1.800
[3,     4] loss: 1.770
[3,     6] loss: 1.755
[3,     8] loss: 1.728
[3,    10] loss: 1.719
[4,     2] loss: 1.702
[4,     4] loss: 1.694
[4,     6] loss: 1.679
[4,     8] loss: 1.668
[4,    10] loss: 1.655
[5,     2] loss: 1.639
[5,     4] loss: 1.631
[5,     6] loss: 1.616
[5,     8] loss: 1.612
[5,    10] loss: 1.597
[6,     2] loss: 1.592
[6,     4] loss: 1.583
[6,     6] loss: 1.578
[6,     8] loss: 1.574
[6,    10] loss: 1.574
[7,     2] loss: 1.566
[7,     4] loss: 1.558
[7,     6] loss: 1.557
[7,     8] loss: 1.553
[7,    10] loss: 1.549
[8,     2] loss: 1.546
[8,     4] loss: 1.544
[8,     6] loss: 1.541
[8,     8] loss: 1.541
[8,    10] loss: 1.538
[9,     2] loss: 1.538
[9,     4] loss: 1.537
[9,     6] loss: 1.533
[9,     8] loss: 1.532
[9,    10] loss: 1.527
[10,     2] loss: 1.530
[10,     4] loss: 1.525
[10,     6] loss: 1.525
[10,    

In [30]:
PATH = './Training Results/03-SAT-In silico.pth'
torch.save(net.state_dict(), PATH)
PATH = ''

### 04-SAT-In situ

In [31]:
from sam import SAM
import torch.optim as optim
net = LeNet5()
net.to(device)
criterion = nn.CrossEntropyLoss()
base_optimizer = torch.optim.Adam
optimizer = SAM(net.parameters(), rho=0.3, base_optimizer=base_optimizer,lr=1e-1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 400, gamma = 0.5)

In [32]:
import time
net.to(device)
start = time.time()
for epoch in range(40):  # loop over the dataset multiple times
### You can consider increasing the training epochs if the accuracy does not converge
### I find the different initialization results may lead to different training epochs, sometimes short, sometimes long
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # forward + backward + optimize
        outputs_sim = net(inputs)
        with torch.no_grad():
            net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_n, dtype=torch.float32).cuda()
            outputs_phy = net.phy_forward(inputs)
            net.phy_replace_sim()
            outputs_sim.data.copy_(outputs_phy.data)
            net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data - b0_phi_error 
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_init, dtype=torch.float32).cuda()   
        loss0 = criterion(outputs_sim, labels)

        loss = loss0
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        
        
        outputs_sim_SAM = net(inputs)
        with torch.no_grad():
            net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_n, dtype=torch.float32).cuda()  
            outputs_phy_SAM = net.phy_forward(inputs)
            net.phy_replace_sim()
            outputs_sim_SAM.data.copy_(outputs_phy_SAM.data)
            net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data - b0_phi_error  
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_init, dtype=torch.float32).cuda()  
        lossSAM0 = criterion(outputs_sim_SAM, labels)

        lossSAM = lossSAM0
        lossSAM.backward()
        optimizer.second_step(zero_grad=True) 
        # net.apply_constraints()
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f}')
            running_loss = 0.0

    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                    
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error 
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda() 
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] loss: 2.314
[1,     4] loss: 2.307
[1,     6] loss: 2.294
[1,     8] loss: 2.291
[1,    10] loss: 2.268
[2,     2] loss: 2.264
[2,     4] loss: 2.238
[2,     6] loss: 2.195
[2,     8] loss: 2.166
[2,    10] loss: 2.191
[3,     2] loss: 2.193
[3,     4] loss: 2.215
[3,     6] loss: 2.150
[3,     8] loss: 2.155
[3,    10] loss: 2.141
[4,     2] loss: 2.137
[4,     4] loss: 2.133
[4,     6] loss: 2.115
[4,     8] loss: 2.152
[4,    10] loss: 2.124
[5,     2] loss: 2.087
[5,     4] loss: 2.071
[5,     6] loss: 2.064
[5,     8] loss: 2.057
[5,    10] loss: 2.061
[6,     2] loss: 2.057
[6,     4] loss: 2.029
[6,     6] loss: 2.035
[6,     8] loss: 2.027
[6,    10] loss: 2.034
[7,     2] loss: 2.036
[7,     4] loss: 2.034
[7,     6] loss: 2.032
[7,     8] loss: 2.028
[7,    10] loss: 2.017
[8,     2] loss: 2.007
[8,     4] loss: 1.986
[8,     6] loss: 1.987
[8,     8] loss: 1.994
[8,    10] loss: 1.993
[9,     2] loss: 1.969
[9,     4] loss: 1.968
[9,     6] loss: 1.992
[9,     8] 

In [33]:
PATH = './Training Results/04-SAT-In situ.pth'
torch.save(net.state_dict(), PATH)
PATH = ''

### 05-DAT (Requires redefining the model)

In [34]:
import torch.nn as nn
import torch.nn.functional as F
import net

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 64)
        self.norm1 = nn.BatchNorm2d(6)
        self.norm2 = nn.BatchNorm2d(16)
        self.norm3 = nn.BatchNorm1d(120)
        self.layer1 = RMTorch(64, phase_error = 0, phase_error_files = None,bs_error=0,bs_error_files = None)
        self.output = CNormSq()
        self.cn1 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])
        self.cn2 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])
        self.cn3 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])
        self.cn4 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])
        self.cn5 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])
        self.cn6 = net.ComplexUNet(size = [8,8],kernel_size = 3, bn_flag=True, CB_layers = [3,3,3], FM_num = [4,8,16])        
    
    def forward(self,x,cn_weight=1.0):

        x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
        x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.norm3(x)
        x = self.fc2(x)
        x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
        x,_,_,_ = self.layer1(x)
        x = (self.cn1(x) + self.cn2(x) + self.cn3(x) + self.cn4(x) + self.cn5(x) + self.cn6(x)) * cn_weight + x
        self.at_sensor = x
        x = self.output(x)
        self.at_sensor_intensity = x
        return self.at_sensor_intensity[:,:10]
    
    def phy_forward(self,x):
        self.in_outs_phy = []
        with torch.no_grad():
            x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
            x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
            x = x.view(x.size(0), -1) 
            x = self.fc1(x)
            x = self.norm3(x)
            x = self.fc2(x)
            x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
            x, _, _, _ = self.layer1(x)
            self.at_sensor_phy = x
            x = self.output(x)
            self.at_sensor_intensity_phy = x
        return self.at_sensor_intensity_phy[:,:10]     
    
    def phy_replace_sim(self):
        with torch.no_grad():
            angle = torch.angle(torch.complex(self.at_sensor[0, :, :], self.at_sensor[1, :, :]))
            amp = torch.abs(torch.complex(self.at_sensor_phy[0, :, :], self.at_sensor_phy[1, :, :]))
            new_data = amp * torch.exp(1j * angle)
            data_transfer = torch.stack([torch.real(new_data), torch.imag(new_data)], dim=0)
            self.at_sensor.data.copy_(data_transfer.data)
            self.at_sensor_intensity.data.copy_(self.at_sensor_intensity_phy.data)             

    def apply_constraints(self):
        self.conv1.cuda().weight.data = torch.clamp(self.conv1.cuda().weight.data, -1, 1)
        self.conv2.cuda().weight.data = torch.clamp(self.conv2.cuda().weight.data, -1, 1)
        self.fc1.cuda().weight.data = torch.clamp(self.fc1.cuda().weight.data, -1, 1)
        self.fc2.cuda().weight.data = torch.clamp(self.fc2.cuda().weight.data, -1, 1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net = LeNet5()
net.to(device)

cuda:0


LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (norm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): RMTorch()
  (output): CNormSq()
  (cn1): ComplexUNet(
    (ConvB1): ComplexConvBlock(
      (ComplexConv-0): ComplexConvLayer(
        (ComplexConv2d): ComplexConv2d(
          (conv_r): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_i): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (ComplexBN): ComplexBatchNorm2d()
        (ComplexAct): ComplexReLU()
      )
      (ComplexConv

In [35]:
[nenn, nepn, nenp, nepp], [enn, epn, enp, epp] = net.layer1.mesh_model.mzi_error_tensors()
[nenn_init, nepn_init, nenp_init, nepp_init] = [nenn, nepn, nenp, nepp]

In [36]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
params = list(net.parameters())
cn1_params = list(net.cn1.parameters())
params_to_optimize = [p for p in params if id(p) not in [id(cn1_p) for cn1_p in cn1_params]]
optimizer = optim.SGD(params_to_optimize, lr=0.1, momentum=0.5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 30, gamma = 0.5)
criterion_cn = nn.MSELoss()
params = list(net.cn1.parameters()) + list(net.cn2.parameters()) + list(net.cn3.parameters()) + list(net.cn4.parameters()) + list(net.cn5.parameters()) + list(net.cn6.parameters())
optimizer_cn = optim.Adam(params, lr=0.001)
scheduler_cn = optim.lr_scheduler.StepLR(optimizer_cn, step_size = 60, gamma = 0.5)

In [37]:
import time
net.to(device)
start = time.time()
for epoch in range(60):  # loop over the dataset multiple times
    running_loss = 0.0
    running_loss_cn = 0.0
    cn_weight = 0. if epoch < 20 else 1. 
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        if epoch < 20:
            optimizer_cn.zero_grad()
            # Optimize sub-network
            outputs_sim = net(inputs)
            net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_n, dtype=torch.float32).cuda()
            outputs_phy = net.phy_forward(inputs)        
            net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data - b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_init, dtype=torch.float32).cuda()  
            
            loss_cn = criterion_cn(outputs_sim, outputs_phy)
            loss_cn.backward()
            optimizer_cn.step()
            scheduler_cn.step()
        # Optimize Main network
        outputs_sim = net(inputs, cn_weight) 
        with torch.no_grad():
            net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_n, dtype=torch.float32).cuda() 
            outputs_phy = net.phy_forward(inputs)
            net.phy_replace_sim()
            outputs_sim.data.copy_(outputs_phy.data)
            net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
            net.layer1.phi.data = net.layer1.phi.data - b0_phi_error
            net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                    torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nepp_init, dtype=torch.float32).cuda() 
        
        loss = criterion(outputs_sim, labels)
        loss.backward()
        optimizer.step()
        net.apply_constraints()
        # print statistics
        running_loss_cn += loss_cn.item()
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] classification loss: {running_loss / 2:.3f} loss_cn: {running_loss_cn / 2:.3f}')
            running_loss = 0.0
            running_loss_cn = 0.0
    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                    
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error 
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda() 
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] classification loss: 2.314 loss_cn: 0.012
[1,     4] classification loss: 2.298 loss_cn: 0.010
[1,     6] classification loss: 2.278 loss_cn: 0.010
[1,     8] classification loss: 2.255 loss_cn: 0.009
[1,    10] classification loss: 2.233 loss_cn: 0.008
[2,     2] classification loss: 2.215 loss_cn: 0.008
[2,     4] classification loss: 2.201 loss_cn: 0.007
[2,     6] classification loss: 2.190 loss_cn: 0.007
[2,     8] classification loss: 2.183 loss_cn: 0.007
[2,    10] classification loss: 2.174 loss_cn: 0.006
[3,     2] classification loss: 2.167 loss_cn: 0.006
[3,     4] classification loss: 2.159 loss_cn: 0.006
[3,     6] classification loss: 2.150 loss_cn: 0.006
[3,     8] classification loss: 2.142 loss_cn: 0.006
[3,    10] classification loss: 2.137 loss_cn: 0.005
[4,     2] classification loss: 2.129 loss_cn: 0.005
[4,     4] classification loss: 2.122 loss_cn: 0.005
[4,     6] classification loss: 2.121 loss_cn: 0.005
[4,     8] classification loss: 2.113 loss_cn:

In [38]:
PATH = './Training Results/05-DAT.pth'
torch.save(net.state_dict(), PATH)
PATH = ''

### 06-NAT

In [50]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 64)
        self.norm1 = nn.BatchNorm2d(6)
        self.norm2 = nn.BatchNorm2d(16)
        self.norm3 = nn.BatchNorm1d(120)
        self.layer1 = RMTorch(64, phase_error = 0, phase_error_files = None,bs_error=0,bs_error_files = None)
        self.output = CNormSq()

    def forward(self,x):

        x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
        x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.norm3(x)
        x = self.fc2(x)
        x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
        x,_,_,_ = self.layer1(x)
        x = self.output(x)
        self.at_sensor_intensity = x
        return self.at_sensor_intensity[:,:10]
    
    def phy_forward(self,x):
        self.in_outs_phy = []
        with torch.no_grad():
            x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2, 2))
            x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), (2, 2))
            x = x.view(x.size(0), -1) 
            x = self.fc1(x)
            x = self.norm3(x)
            x = self.fc2(x)
            x = torch.stack((x, torch.zeros(x.shape, dtype=torch.float32, device=x.device)), dim=0)
            x, _, _, _ = self.layer1(x)
            x = self.output(x)
            self.at_sensor_intensity_phy = x
        return self.at_sensor_intensity_phy[:,:10]    
    
    def phy_replace_sim(self):
        # PAT: replace the output
        with torch.no_grad():
            self.at_sensor_intensity.data.copy_(self.at_sensor_intensity_phy.data)    

    def apply_constraints(self):
        self.conv1.cuda().weight.data = torch.clamp(self.conv1.cuda().weight.data, -1, 1)
        self.conv2.cuda().weight.data = torch.clamp(self.conv2.cuda().weight.data, -1, 1)
        self.fc1.cuda().weight.data = torch.clamp(self.fc1.cuda().weight.data, -1, 1)
        self.fc2.cuda().weight.data = torch.clamp(self.fc2.cuda().weight.data, -1, 1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net = LeNet5()
net.to(device)

cuda:0


LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (norm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): RMTorch()
  (output): CNormSq()
)

In [51]:
import torch.optim as optim
net = LeNet5()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 400, gamma = 0.5)

In [52]:
import time
net.to(device)
start = time.time()
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        # Add noise
        net.layer1.mesh_model.bs_error = 0.01
        [nenn, nepn, nenp, nepp], [enn, epn, enp, epp] = net.layer1.mesh_model.mzi_error_tensors()
        net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                            torch.as_tensor(enn, dtype=torch.float32).cuda(), \
                                torch.as_tensor(epn, dtype=torch.float32).cuda(), \
                            torch.as_tensor(enp, dtype=torch.float32).cuda(), \
                                torch.as_tensor(epp, dtype=torch.float32).cuda()
        b0_theta_error_random = torch.randn(net.layer1.theta.shape).cuda() * 0.01
        b0_phi_error_random = torch.randn(net.layer1.phi.shape).cuda() * 0.01
        outputs_sim = net(inputs)

        net.layer1.theta.data = net.layer1.theta.data - b0_theta_error_random
        net.layer1.phi.data = net.layer1.phi.data - b0_phi_error_random  
        net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                    torch.as_tensor(nepp_init, dtype=torch.float32).cuda() 

        loss0 = criterion(outputs_sim, labels)
        loss = loss0
        loss.backward()
        optimizer.step()
        net.apply_constraints()
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f} Classification loss:{loss0.cpu().detach().numpy():.3f}')
            running_loss = 0.0
    scheduler.step()
    if epoch % 10 == 9:    # print every 2000 mini-batches
        print(f'epoch:{epoch}')
        net.to(device)
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                net.layer1.theta.data = net.layer1.theta.data + b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data + b0_phi_error
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_n, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_n, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_n, dtype=torch.float32).cuda()                
                outputs = net.phy_forward(inputs)
                net.layer1.theta.data = net.layer1.theta.data - b0_theta_error
                net.layer1.phi.data = net.layer1.phi.data - b0_phi_error  
                net.layer1.nenn, net.layer1.nepn, net.layer1.nenp, net.layer1.nepp = \
                        torch.as_tensor(nenn_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepn_init, dtype=torch.float32).cuda(), \
                        torch.as_tensor(nenp_init, dtype=torch.float32).cuda(), \
                            torch.as_tensor(nepp_init, dtype=torch.float32).cuda()   
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy 10000 test images: {100 * correct / total} %')
        accu_wo_n = 100 * correct / total
print('Finished Training')
end = time.time()
print(end-start)

[1,     2] loss: 2.289 Classification loss:2.272
[1,     4] loss: 2.185 Classification loss:2.153
[1,     6] loss: 2.061 Classification loss:2.026
[1,     8] loss: 1.957 Classification loss:1.942
[1,    10] loss: 1.878 Classification loss:1.865
[2,     2] loss: 1.824 Classification loss:1.811
[2,     4] loss: 1.784 Classification loss:1.776
[2,     6] loss: 1.763 Classification loss:1.760
[2,     8] loss: 1.740 Classification loss:1.734
[2,    10] loss: 1.718 Classification loss:1.713
[3,     2] loss: 1.703 Classification loss:1.699
[3,     4] loss: 1.688 Classification loss:1.685
[3,     6] loss: 1.678 Classification loss:1.671
[3,     8] loss: 1.666 Classification loss:1.666
[3,    10] loss: 1.656 Classification loss:1.654
[4,     2] loss: 1.648 Classification loss:1.648
[4,     4] loss: 1.639 Classification loss:1.636
[4,     6] loss: 1.630 Classification loss:1.628
[4,     8] loss: 1.627 Classification loss:1.624
[4,    10] loss: 1.620 Classification loss:1.619
[5,     2] loss: 1.6

In [53]:
PATH = './Training Results/06-NAT.pth'
torch.save(net.state_dict(), PATH)
PATH = ''